In [2]:
# import
import requests
from urllib.parse import urlparse
import urllib.parse
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import re
import seaborn as sns
import json
import folium
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=False)
import sqlite3

In [3]:
key_fd = open('./과제data/gov_data_api_key.txt', mode='r')
govapi_key = key_fd.read(100)
key_fd.close()

In [4]:
start_date = '20200101'
end_date = datetime.today().strftime("%Y%m%d")

### 시도발생현황

In [104]:
corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
url = f'{corona_url}?ServiceKey={govapi_key}&pageNo=1&numOfRows=10&startCreateDt={start_date}&endCreateDt={end_date}'

In [105]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
soup.find('resultCode').get_text()

'00'

In [106]:
url

'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?ServiceKey=QdYry9tFcAehmt%2BH6Q2LYgChFxQ%2BMZu1oMldd711rMJLoU%2FNOZ61YLzvo01MFZJSElrRYZSRwyDwO6M7Bsmikw%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200101&endCreateDt=20201224'

In [107]:
createDt_list,deathCnt_list,defCnt_list,gubun_list,incDec_list = [],[],[],[],[]
isolClearCnt_list,isolIngCnt_list,localOccCnt_list,overFlowCnt_list = [],[],[],[]
seq_list,qurRate_list,stdDay_list,updateDt_list = [],[],[],[]
items = soup.find_all('item')
for item in items:
    createDt_list.append(item.find('createDt').string if item.find('createDt') else '')
    deathCnt_list.append(item.find('deathCnt').string if item.find('deathCnt') else '')
    defCnt_list.append(item.find('defCnt').string if item.find('defCnt') else 0)
    gubun_list.append(item.find('gubun').string if item.find('gubun') else '')
    incDec_list.append(item.find('incDec').string if item.find('incDec') else '')
    isolClearCnt_list.append(item.find('isolClearCnt').string if item.find('isolClearCnt') else '')
    isolIngCnt_list.append(item.find('isolIngCnt').string if item.find('isolIngCnt') else '')
    localOccCnt_list.append(item.find('localOccCnt').string if item.find('localOccCnt') else '')
    overFlowCnt_list.append(item.find('overFlowCnt').string if item.find('overFlowCnt') else '')
    qurRate_list.append(item.find('qurRate').string if item.find('qurRate') else '')
    stdDay_list.append(item.find('stdDay').string if item.find('stdDay') else '')
    updateDt_list.append(item.find('updateDt').string if item.find('updateDt') else '')
    seq_list.append(item.find('seq').string)

df = pd.DataFrame({
    '등록시간':createDt_list, '사망자':deathCnt_list, '확진자':defCnt_list,
    '광역시도':gubun_list, '전일대비':incDec_list, '격리해제':isolClearCnt_list, 
    '격리중':isolIngCnt_list, '지역발생':localOccCnt_list,'해외유입':overFlowCnt_list,
    '10만명당':qurRate_list, '기준시간':stdDay_list, '수정시간':updateDt_list, 'seq' : seq_list
})
df


,등록시간,사망자,확진자,광역시도,전일대비,격리해제,격리중,지역발생,해외유입,10만명당,기준시간,수정시간,seq
0,2020-12-24 09:45:57.88,1,2381,검역,11,1661,719,0,11,-,2020년 12월 24일 00시,null,6600
1,2020-12-24 09:45:57.88,0,330,제주,26,116,214,26,0,49.20,2020년 12월 24일 00시,null,6599
2,2020-12-24 09:45:57.879,3,1145,경남,21,811,331,21,0,34.06,2020년 12월 24일 00시,null,6598
3,2020-12-24 09:45:57.879,61,2169,경북,50,1742,366,50,0,81.46,2020년 12월 24일 00시,null,6597
4,2020-12-24 09:45:57.879,5,527,전남,2,450,72,2,0,28.26,2020년 12월 24일 00시,null,6596
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5947,2020-03-02 19:27:57.57,0,0,인천,0,,,,,None,2020년 3월 1일 16시,None,5
5948,2020-03-02 19:27:57.57,9,0,대구,469,,,,,None,2020년 3월 1일 16시,None,4
5949,2020-03-02 19:27:57.57,0,0,부산,3,,,,,None,2020년 3월 1일 16시,None,3
5950,2020-03-02 19:27:57.57,0,0,서울,10,,,,,None,2020년 3월 1일 16시,None,2


In [108]:
df = df[['기준시간', '광역시도', '확진자', '사망자', '전일대비', '격리해제', '격리중', '지역발생', '해외유입', '10만명당','seq']]

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5952 entries, 0 to 5951
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기준시간    5952 non-null   object
 1   광역시도    5952 non-null   object
 2   확진자     5952 non-null   object
 3   사망자     5952 non-null   object
 4   전일대비    5952 non-null   object
 5   격리해제    5952 non-null   object
 6   격리중     5952 non-null   object
 7   지역발생    5952 non-null   object
 8   해외유입    5952 non-null   object
 9   10만명당   5933 non-null   object
 10  seq     5952 non-null   object
dtypes: object(11)
memory usage: 511.6+ KB


In [110]:
df['일자'] = df['기준시간'].apply(lambda r : r.split('일')[0].replace('년 ','-').replace('월 ','-'))

In [111]:
df['일자']

0       2020-12-24
1       2020-12-24
2       2020-12-24
3       2020-12-24
4       2020-12-24
           ...    
5947      2020-3-1
5948      2020-3-1
5949      2020-3-1
5950      2020-3-1
5951      2020-3-1
Name: 일자, Length: 5952, dtype: object

In [112]:
df['일자'] = pd.to_datetime(df['일자']) 

In [113]:
df['일자2'] = df['일자'].astype(str)

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5952 entries, 0 to 5951
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   기준시간    5952 non-null   object        
 1   광역시도    5952 non-null   object        
 2   확진자     5952 non-null   object        
 3   사망자     5952 non-null   object        
 4   전일대비    5952 non-null   object        
 5   격리해제    5952 non-null   object        
 6   격리중     5952 non-null   object        
 7   지역발생    5952 non-null   object        
 8   해외유입    5952 non-null   object        
 9   10만명당   5933 non-null   object        
 10  seq     5952 non-null   object        
 11  일자      5952 non-null   datetime64[ns]
 12  일자2     5952 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 604.6+ KB


In [115]:
for i in df['seq'].unique():
    if len(df[df['seq'] == i]) > 1 :
        x = df[df['seq'] == i][1:].index
        for k in x :
            df.drop(k, inplace=True)

In [116]:
for i in df['일자2'].unique():
    if len(df[df['일자2'] == i]) > 20:
        x = df[df['일자2'] == i].tail(len(df[df['일자2'] == i]) - 19).index
        for k in x:
            df.drop(k, inplace=True)

In [117]:
df[df['일자2'] == '2020-12-21']

,기준시간,광역시도,확진자,사망자,전일대비,격리해제,격리중,지역발생,해외유입,10만명당,seq,일자,일자2
138,2020년 12월 21일 00시,검역,2343,1,10,1658,684,0,10,-,6467,2020-12-21,2020-12-21
139,2020년 12월 21일 00시,제주,252,0,23,102,150,23,0,18.93,6466,2020-12-21,2020-12-21
141,2020년 12월 21일 00시,경남,1077,3,15,748,326,15,0,26.36,6465,2020-12-21,2020-12-21
143,2020년 12월 21일 00시,경북,2035,61,48,1715,259,48,0,70.05,6464,2020-12-21,2020-12-21
145,2020년 12월 21일 00시,전남,508,5,9,418,85,8,1,25.79,6463,2020-12-21,2020-12-21
147,2020년 12월 21일 00시,전북,688,7,13,441,240,11,2,28.67,6462,2020-12-21,2020-12-21
149,2020년 12월 21일 00시,충남,1288,18,16,950,320,16,0,54.47,6461,2020-12-21,2020-12-21
151,2020년 12월 21일 00시,충북,803,13,33,424,366,29,4,36.26,6460,2020-12-21,2020-12-21
153,2020년 12월 21일 00시,강원,976,13,25,692,271,22,3,54.14,6459,2020-12-21,2020-12-21
155,2020년 12월 21일 00시,경기,11946,173,244,8235,3538,237,7,77.36,6458,2020-12-21,2020-12-21


In [118]:
len(df[df['일자2'] == '2020-12-17'])

19

In [119]:
df.head()

,기준시간,광역시도,확진자,사망자,전일대비,격리해제,격리중,지역발생,해외유입,10만명당,seq,일자,일자2
0,2020년 12월 24일 00시,검역,2381,1,11,1661,719,0,11,-,6600,2020-12-24,2020-12-24
1,2020년 12월 24일 00시,제주,330,0,26,116,214,26,0,49.20,6599,2020-12-24,2020-12-24
2,2020년 12월 24일 00시,경남,1145,3,21,811,331,21,0,34.06,6598,2020-12-24,2020-12-24
3,2020년 12월 24일 00시,경북,2169,61,50,1742,366,50,0,81.46,6597,2020-12-24,2020-12-24
4,2020년 12월 24일 00시,전남,527,5,2,450,72,2,0,28.26,6596,2020-12-24,2020-12-24


In [120]:
df.sort_index(ascending=False, inplace=True)

In [121]:
df.head()

,기준시간,광역시도,확진자,사망자,전일대비,격리해제,격리중,지역발생,해외유입,10만명당,seq,일자,일자2
5951,2020년 3월 1일 16시,합계,0,18,586,,,,,None,1,2020-03-01,2020-03-01
5950,2020년 3월 1일 16시,서울,0,0,10,,,,,None,2,2020-03-01,2020-03-01
5949,2020년 3월 1일 16시,부산,0,0,3,,,,,None,3,2020-03-01,2020-03-01
5948,2020년 3월 1일 16시,대구,0,9,469,,,,,None,4,2020-03-01,2020-03-01
5947,2020년 3월 1일 16시,인천,0,0,0,,,,,None,5,2020-03-01,2020-03-01


In [122]:
df = df[['seq','일자2','광역시도','확진자','사망자','전일대비','격리해제','격리중','지역발생','해외유입','10만명당']]

In [123]:
df.reset_index(drop=True, inplace=True)
df.head()

,seq,일자2,광역시도,확진자,사망자,전일대비,격리해제,격리중,지역발생,해외유입,10만명당
0,1,2020-03-01,합계,0,18,586,,,,,None
1,2,2020-03-01,서울,0,0,10,,,,,None
2,3,2020-03-01,부산,0,0,3,,,,,None
3,4,2020-03-01,대구,0,9,469,,,,,None
4,5,2020-03-01,인천,0,0,0,,,,,None


In [124]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [125]:
sql_table = '''
create table '시도발생_현황'(
       'seq' text not null primary key, '일자' text not null,
       '광역시도' text not null,'확진자' int default 0, '사망자' int default 0,
       '전일대비' int default 0, '격리해제' int default 0, '격리중' int default 0,
       '지역발생' int default 0, '해외유입' int default 0, '10만명당' text default '-'
       
)
'''

In [126]:
cur.execute(sql_table)
conn.commit()

In [127]:
sql_insert = 'insert into "시도발생_현황" values(?,?,?,?,?,?,?,?,?,?,?)'

In [128]:
for i in df.index:
    params = list(df.loc[i])
    cur.execute(sql_insert, params)
    conn.commit()

In [129]:
conn.close()

In [130]:
df.to_csv('./DB/시도발생현황.csv')

In [131]:
df['확진자'] = df['확진자'].astype(int)

In [132]:
a_df = df.pivot_table('확진자', index=['일자2', '광역시도'])

In [133]:
a_df

확진자
일자2        광역시도       
2020-03-01 강원        0
           검역        0
           경기        0
           경남        0
           경북        0
...                ...
2020-12-24 전북      725
           제주      330
           충남     1347
           충북      939
           합계    53533

[5593 rows x 1 columns]

In [134]:
a_df.to_csv('./DB/광역시도별.csv')

In [135]:
n_df = df.pivot_table('확진자', '일자2', '광역시도')

In [136]:
day_df = n_df - n_df.shift(1)

In [137]:
day_df.fillna(0,inplace=True)

In [138]:
del day_df['검역']

In [139]:
day_df.astype(int)

광역시도,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북,합계
일자2,,,,,,,,,,,,,,,,,,
2020-03-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-04,21,101,65,774,13,4006,15,93,99,1,23,9,5,7,3,82,11,5328
2020-03-05,2,9,9,87,1,321,1,-1,4,0,0,0,-1,0,1,4,1,438
2020-03-06,2,10,3,123,-1,367,2,3,2,0,0,0,0,0,0,4,3,518
2020-03-07,1,10,5,65,0,390,0,1,3,1,0,0,0,0,0,2,5,483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-20,55,249,51,25,14,25,3,33,471,1,6,62,4,29,24,20,14,1095
2020-12-21,25,244,15,48,13,21,3,20,328,0,17,88,9,13,23,16,33,926
2020-12-22,23,206,18,59,26,39,9,26,317,1,6,45,5,4,19,17,31,869


In [140]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [141]:
sql_table = '''
create table '일일확진자 현황'(
       '일자' text not null primary key,
       '강원' int not null, '경기' int not null, '경남' int not null,
       '경북' int not null, '광주' int not null, '대구' int not null,
       '대전' int not null, '부산' int not null, '서울' int not null,
       '세종' int not null, '울산' int not null, '인천' int not null,
       '전남' int not null, '전북' int not null, '제주' int not null,
       '충남' int not null, '충북' int not null, '합계' int not null
)
'''

In [142]:
cur.execute(sql_table)
conn.commit()

In [143]:
sql_insert = 'insert into "일일확진자 현황" values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'

In [144]:
for i in day_df[1:].index:
    params = list(day_df.loc[i])
    params.insert(0, i)
    cur.execute(sql_insert, params)
    conn.commit()

In [145]:
conn.close()

In [146]:
day_df.to_csv('./DB/일일확진자.csv')

### 연령별 현황

In [5]:
corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?serviceKey='
url = f'{corona_url}{govapi_key}&pageNo=1&numOfRows=10&startCreateDt={start_date}&endCreateDt={end_date}'

In [6]:
url

'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?serviceKey=QdYry9tFcAehmt%2BH6Q2LYgChFxQ%2BMZu1oMldd711rMJLoU%2FNOZ61YLzvo01MFZJSElrRYZSRwyDwO6M7Bsmikw%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200101&endCreateDt=20201228'

In [148]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
soup.find('resultCode').get_text()

'00'

In [149]:
confCase_list, confCaseRate_list, createDt_list, criticalRate_list = [],[],[],[]
death_list, deathRate_list, gubun_list, id_list, updateDt_list =[],[],[],[],[]
items = soup.find_all('item')
for item in items:
    id_list.append(item.find('seq').string)
    createDt_list.append(item.find('createDt').string.split(' ')[0])
    confCase_list.append(item.find('confCase').string)
    confCaseRate_list.append(item.find('confCaseRate').string)
    criticalRate_list.append(item.find('criticalRate').string)
    death_list.append(item.find('death').string)
    deathRate_list.append(item.find('deathRate').string)
    gubun_list.append(item.find('gubun').string)
    updateDt_list.append(item.find('updateDt').string if item.find('updateDt') else ' ')

df = pd.DataFrame({
    'rowID':id_list,
    '등록일':createDt_list, 
    '확진자': confCase_list,
    '확진률' : confCaseRate_list,
    '치명률' : criticalRate_list,
    '사망자' : death_list,
    '사망률' : deathRate_list, 
    '구분' : gubun_list,
    '수정일' :  updateDt_list
})
df

,rowID,등록일,확진자,확진률,치명률,사망자,사망률,구분,수정일
0,5609,2020-12-23,1761,3.35,0,0,0.00,0-9,null
1,5608,2020-12-23,3268,6.22,0,0,0.00,10-19,null
2,5607,2020-12-23,8758,16.67,0,0,0.00,20-29,null
3,5606,2020-12-23,6642,12.64,0.05,3,0.41,30-39,null
4,5605,2020-12-23,7367,14.02,0.08,6,0.81,40-49,null
...,...,...,...,...,...,...,...,...,...
2845,5,2020-04-02,1235,12.62,1.78,22,13.58,60-69,None
2846,4,2020-04-02,651,6.65,7.07,46,28.4,70-79,None
2847,3,2020-04-02,442,4.52,18.55,82,50.62,80 이상,None
2848,2,2020-04-02,5881,60.1,1.36,80,49.38,여성,None


In [150]:
df= df[['rowID', '등록일', '구분', '확진자', '확진률', '치명률', '사망자', '사망률', '수정일']]

In [151]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [152]:
sql_table = '''
create table '연령별·성별감염_현황'(
       'rowID' text not null primary key, '등록일' text not null, '구분' text not null,
       '확진자' int not null, '확진률' int not null, '치명률' float default 0,
       '사망자' int default 0, '사망률' int default 0, 
       '수정일' int default '-'
       )
'''

In [153]:
cur.execute(sql_table)
conn.commit()

In [154]:
len(df.columns)

9

In [155]:
sql_insert = 'insert into "연령별·성별감염_현황" values(?,?,?,?,?,?,?,?,?)'

In [156]:
for i in range(len(df)):
    params = list(df.loc[i])
    cur.execute(sql_insert, params)
    conn.commit()

In [157]:
conn.close()

In [158]:
df.to_csv('./DB/연령별,성별감염_현황.csv')

###  코로나19감염증으로 인한 일별 확진자, 완치자, 치료중인환자, 사망자등에 대한 현황자료

In [159]:
corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
url = f'{corona_url}?ServiceKey={govapi_key}&pageNo=1&numOfRows=10&startCreateDt={start_date}&endCreateDt={end_date}'

In [160]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
soup.find('resultCode').get_text()

'99'

In [161]:
url

'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?ServiceKey=QdYry9tFcAehmt%2BH6Q2LYgChFxQ%2BMZu1oMldd711rMJLoU%2FNOZ61YLzvo01MFZJSElrRYZSRwyDwO6M7Bsmikw%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200101&endCreateDt=20201224'

In [162]:
accDefRate_list, accExamCnt_list, accExamCompCnt_list, careCnt_list = [],[],[],[]
clearCnt_list, deathCnt_list, decideCnt_list, examCnt_list = [],[],[],[]
resutlNegCnt_list, seq_list, stateDt_list, updateDt_list = [],[],[],[]
items = soup.find_all('item')
for item in items:
    seq_list.append(item.find('seq').string)
    stateDt_list.append(item.find('stateDt').string)
    accDefRate_list.append(item.find('accDefRate').string if item.find('accDefRate') else '')
    accExamCnt_list.append(item.find('accExamCnt').string if item.find('accExamCnt') else '')
    accExamCompCnt_list.append(item.find('accExamCompCnt').string if item.find('accExamCompCnt') else '')
    careCnt_list.append(item.find('careCnt').string if item.find('careCnt') else '')
    clearCnt_list.append(item.find('clearCnt').string if item.find('clearCnt') else '')
    deathCnt_list.append(item.find('deathCnt').string if item.find('deathCnt') else '')
    decideCnt_list.append(item.find('decideCnt').string if item.find('decideCnt') else '')
    examCnt_list.append(item.find('examCnt').string if item.find('examCnt') else '')
    resutlNegCnt_list.append(item.find('resutlNegCnt').string if item.find('resutlNegCnt') else '')
    updateDt_list.append(item.find('updateDt').string if item.find('updateDt') else '')

In [163]:
df = pd.DataFrame({
    'rowID': seq_list,
    '기준일': stateDt_list, 
    '누적환진률': accDefRate_list,
    '누적검사수' : accExamCnt_list,
    '누적 검사 완료 수' : accExamCompCnt_list,
    '치료중 환자 수' : careCnt_list,
    '격리해제 수' : clearCnt_list, 
    '사망자 수' : deathCnt_list,
    '확진자 수' :  decideCnt_list,
    '검사 진행 수' : examCnt_list,
    '결과 음성 수 ': resutlNegCnt_list,
    '수정일' : updateDt_list
})
df

,rowID,기준일,누적환진률,누적검사수,누적 검사 완료 수,치료중 환자 수,격리해제 수,사망자 수,확진자 수,검사 진행 수,결과 음성 수,수정일


In [164]:
# 중복자료 지우기
for i in df['rowID'].unique():
    if len(df[df['rowID'] == i]) > 1:
        for k in df[df['rowID'] == i][1:].index:
            df.drop(k, inplace=True)

In [165]:
df.reset_index(drop=True,inplace=True)

In [166]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [167]:
sql_table = '''
create table '감염_현황'(
    'rowID' text not null primary key, '기준일' text not null,
    '누적환진률' flaot,
    '누적검사수' text, '누적 검사 완료 수' text, '치료중 환자 수' text,
    '격리해제 수' text,   '사망자 수' text , '확진자 수' text,
    '검사 진행 수' text, '결과 음성 수' text, '수정일' text 
    )
'''

In [168]:
cur.execute(sql_table)
conn.commit()

In [169]:
sql_insert = 'insert into "감염_현황" values(?,?,?,?,?,?,?,?,?,?,?,?)'

In [170]:
for i in range(len(df)):
    params = list(df.loc[i])
    cur.execute(sql_insert, params)
    conn.commit()

In [171]:
conn.close()

In [172]:
df.to_csv('./DB/감염현황.csv')